# Gather only NCI-funded grants from NIH RePORTER API
Started 2023-10-02 ZD  

See [INS-767](https://tracker.nci.nih.gov/browse/INS-767) for details

In [9]:
import pandas as pd
import requests
from datetime import datetime
from time import sleep # for retrying API calls
from math import ceil # for pagination logging

# Method to import from parent directory
import os
import sys
root_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))
sys.path.append(root_dir)

import config

In [17]:
def check_agency_ic_admins(grants_data:str):
    """Check raw json API results for unique ICs and counts"""

    adminList = []

    for grant in grants_data:
        admin = grant['agency_ic_admin']['code']
        adminList.append(admin)

    ic_counts = pd.Series(adminList).value_counts()

    return ic_counts

In [2]:
from modules import nih_reporter_api
                    # "exclude_subprojects": True,
                    # "agencies": ["NCI"],
                    # #"is_agency_funding": True,
                    # "fiscal_years": fiscal_year_list


grants_data = nih_reporter_api.get_nih_reporter_grants(["PA20-185"], "nofo", True)


nofo: PA20-185 (1/6): {'search_id': 'mepQqEI8okyvjbkQNGiIxg', 'total': 2905, 'offset': 0, 'limit': 500, 'sort_field': 'FiscalYear', 'sort_order': 'desc', 'sorted_by_relevance': False, 'properties': {'URL': 'https:/reporter.nih.gov/search/mepQqEI8okyvjbkQNGiIxg/projects'}}
nofo: PA20-185 (2/6): {'search_id': 'sSP-ZT5L_EeysHuT79Dwyw', 'total': 2905, 'offset': 500, 'limit': 500, 'sort_field': 'FiscalYear', 'sort_order': 'desc', 'sorted_by_relevance': False, 'properties': {'URL': 'https:/reporter.nih.gov/search/sSP-ZT5L_EeysHuT79Dwyw/projects'}}
nofo: PA20-185 (3/6): {'search_id': 'tan9UtpBpEGqQwYTE-nxSg', 'total': 2905, 'offset': 1000, 'limit': 500, 'sort_field': 'FiscalYear', 'sort_order': 'desc', 'sorted_by_relevance': False, 'properties': {'URL': 'https:/reporter.nih.gov/search/tan9UtpBpEGqQwYTE-nxSg/projects'}}
nofo: PA20-185 (4/6): {'search_id': '6X1j5Tj0Tkut3XKmIxpGIg', 'total': 2905, 'offset': 1500, 'limit': 500, 'sort_field': 'FiscalYear', 'sort_order': 'desc', 'sorted_by_relevanc

In [4]:
check_agency_ic_admins(grants_data)

CA    2905
Name: count, dtype: int64

In [10]:
def get_nih_reporter_grants(search_values:str, 
                            search_type:str, 
                            print_meta=False):
    """Get grants info for either NOFOs or Awards as specified.
    
    :param search_values: string of values to search, separated by semicolons

    :param search_type: type of search (e.g., 'nofo' or 'award')

    :param print_meta: boolean indicator. If True, print API gathering 
                        process results to console.
    """

    # Set the search field based on the search_type
    if search_type == 'nofo':
        search_field = "opportunity_numbers"
    elif search_type == 'award':
        search_field = "project_nums"
    else:
        raise ValueError("Invalid search type.")

    base_url = "https://api.reporter.nih.gov/v2/projects/search"
    grants_data = []

    for search_value in search_values:
        # Check for blank search values
        if not search_value:
            print(f"Blank {search_type} value encountered. Skipping.")
            continue

        # Set default values for params not likley to change
        LIMIT = 500
        MAX_ATTEMPTS = 5
        RETRY_TIME = 2
        EARLIEST_FISCAL_YEAR = config.API_EARLIEST_FISCAL_YEAR

        # Get list of fiscal years to query
        current_year = datetime.now().year
        fiscal_year_list = [str(year) for year in range(
            EARLIEST_FISCAL_YEAR, current_year + 1)]

        # Set starting value for counters
        offset = 0
        page = 0
        attempts = 0

        # RePORTER API sets a max limit of 500 records per call.
        # Keep looping each call in "pages" until the number of records 
        # gathered reaches the total number of records available. 
        
        # Set a cap on the number of attempts at a failed call before 
        # moving on to the next award.
        while attempts < MAX_ATTEMPTS:
            # Set parameters for API call
            params = {
                "criteria": {
                    search_field: [search_value],
                    "exclude_subprojects": True,
                    "agencies": ["NCI"],
                    "is_agency_funding": True,
                    "fiscal_years": fiscal_year_list
                },
                "limit": LIMIT,
                "offset": offset,
                "sort_field": "FiscalYear",
                "sort_order": "desc",
            }

            try: 
                # Define response details
                response = requests.post(base_url, 
                                        json=params, 
                                        headers={
                                            "accept": "application/json", 
                                            "Content-Type": "application/json"})

                # If response is good, get results
                if response.status_code == 200:
                    grants = response.json()
                    # Add API source indicator
                    for grant in grants['results']:
                        grant['api_source_search'] = f"{search_type}_{search_value}"
                    # Add grants to running list
                    grants_data.extend(grants['results'])

                    # Increase offset by limit to get next "page"
                    total_records = grants['meta']['total']
                    offset = offset + LIMIT
                    page = page + 1

                    # Print paginated partial optional metadata
                    # Consider replacing this with proper logging
                    if print_meta == True:
                        total_pages = max(ceil(total_records/LIMIT),1)
                        print(f"{search_type}: {search_value} "
                              f"({page}/{total_pages}): {grants['meta']}")

                    # Stop looping if offset has reached total record count
                    if offset >= total_records:
                        break

                # Handle 500 errors by retrying after 2 second delay
                elif response.status_code == 500:
                    attempts = attempts + 1
                    print(f"Received a 500 error for "
                          f"{search_type} '{search_value}'. "
                          f"Retrying after {RETRY_TIME} seconds. "
                          f"Attempt {attempts}/{MAX_ATTEMPTS}")
                    sleep(RETRY_TIME)
                else:
                    print(f"Error occurred while fetching grants for "
                          f"{search_type} '{search_value}': "
                          f"{response.status_code}")
                    break

            except requests.exceptions.RequestException as e:
                print(f"An error occurred while making the API call for "
                      f"{search_type} '{search_value}': {e}")
                break

    return grants_data

In [13]:
grants_data = get_nih_reporter_grants(["PA20-185"], "nofo", True)
check_agency_ic_admins(grants_data)

nofo: PA20-185 (1/6): {'search_id': 'u6HQzIwOTkymJfDzRFLJvA', 'total': 2907, 'offset': 0, 'limit': 500, 'sort_field': 'FiscalYear', 'sort_order': 'desc', 'sorted_by_relevance': False, 'properties': {'URL': 'https:/reporter.nih.gov/search/u6HQzIwOTkymJfDzRFLJvA/projects'}}
nofo: PA20-185 (2/6): {'search_id': '_yGpGMu6tUSITLX7JBmP9A', 'total': 2907, 'offset': 500, 'limit': 500, 'sort_field': 'FiscalYear', 'sort_order': 'desc', 'sorted_by_relevance': False, 'properties': {'URL': 'https:/reporter.nih.gov/search/_yGpGMu6tUSITLX7JBmP9A/projects'}}
nofo: PA20-185 (3/6): {'search_id': '-eWmMlyfWE2Rd_vXZ9EA6Q', 'total': 2907, 'offset': 1000, 'limit': 500, 'sort_field': 'FiscalYear', 'sort_order': 'desc', 'sorted_by_relevance': False, 'properties': {'URL': 'https:/reporter.nih.gov/search/-eWmMlyfWE2Rd_vXZ9EA6Q/projects'}}
nofo: PA20-185 (4/6): {'search_id': '1iDWMufSkkeGwHaLsvXKgg', 'total': 2907, 'offset': 1500, 'limit': 500, 'sort_field': 'FiscalYear', 'sort_order': 'desc', 'sorted_by_relevanc

CA    2905
HD       2
Name: count, dtype: int64

In [14]:
from modules import clean_grants_data

# STEP 3: Data Cleaning - Clean and format grants data

# Run cleaning steps
cleaned_grants_data = clean_grants_data.clean_grants_data(grants_data)

# Add program id column from alphanumeric program name
program_id = ''.join(filter(str.isalnum, "All of Us"))
cleaned_grants_data[config.PROGRAM_ID_FIELDNAME] = program_id

2 grants without NCI funding removed from list. 
2905 NCI-funded grants remain.


In [19]:
df = pd.DataFrame(grants_data)

In [20]:
df.columns.tolist()

['appl_id',
 'subproject_id',
 'fiscal_year',
 'project_num',
 'project_serial_num',
 'organization',
 'award_type',
 'activity_code',
 'award_amount',
 'is_active',
 'project_num_split',
 'principal_investigators',
 'contact_pi_name',
 'program_officers',
 'agency_ic_admin',
 'agency_ic_fundings',
 'cong_dist',
 'spending_categories',
 'project_start_date',
 'project_end_date',
 'organization_type',
 'opportunity_number',
 'full_study_section',
 'award_notice_date',
 'is_new',
 'mechanism_code_dc',
 'core_project_num',
 'terms',
 'pref_terms',
 'abstract_text',
 'project_title',
 'phr_text',
 'spending_categories_desc',
 'agency_code',
 'covid_response',
 'arra_funded',
 'budget_start',
 'budget_end',
 'cfda_code',
 'funding_mechanism',
 'direct_cost_amt',
 'indirect_cost_amt',
 'project_detail_url',
 'date_added',
 'api_source_search']

In [22]:
def get_admin_agency_col(df):
    """Extract admin agency from nested JSON field"""

    # Extract defined org subfields as new columns and fill with values
    df['admin_agency'] = (df['agency_ic_admin']
                        .apply(lambda row: row['admin_agency']))
    # Drop the old nested organization column after extracting contents
    df.drop('agency_ic_admin', axis=1, inplace=True)
    
    return df

In [32]:
df_admins = df.copy()

df_admins['agency_ic_admin'][0]

{'code': 'CA', 'abbreviation': 'NCI', 'name': 'National Cancer Institute'}

In [37]:
df_admins['admin_agency'] = [admin.get('code') for admin in df['agency_ic_admin']]

In [47]:
df_admins[df_admins['admin_agency'] != 'CA']

,appl_id,subproject_id,fiscal_year,project_num,project_serial_num,organization,award_type,activity_code,award_amount,is_active,...,budget_start,budget_end,cfda_code,funding_mechanism,direct_cost_amt,indirect_cost_amt,project_detail_url,date_added,api_source_search,admin_agency
1429,10698079,None,2023,5R01HD109707-02,HD109707,{'org_name': 'INDIANA UNIV-PURDUE UNIV AT INDI...,5,R01,657190,True,...,2023-07-01T12:07:00Z,2024-06-30T12:06:00Z,865,Non-SBIR/STTR,414631,242559,https://reporter.nih.gov/project-details/10698079,2023-07-22T04:07:20Z,nofo_PA20-185,HD
2125,10512390,None,2022,1R01HD109707-01,HD109707,{'org_name': 'INDIANA UNIV-PURDUE UNIV AT INDI...,1,R01,657190,False,...,2022-09-15T12:09:00Z,2023-06-30T12:06:00Z,865,Non-SBIR/STTR,414631,242559,https://reporter.nih.gov/project-details/10512390,2022-09-17T04:09:14Z,nofo_PA20-185,HD


In [46]:
(df_admins[df_admins['admin_agency'] != 'CA']).iloc[0]

appl_id                                                              10698079
subproject_id                                                            None
fiscal_year                                                              2023
project_num                                                   5R01HD109707-02
project_serial_num                                                   HD109707
organization                {'org_name': 'INDIANA UNIV-PURDUE UNIV AT INDI...
award_type                                                                  5
activity_code                                                             R01
award_amount                                                           657190
is_active                                                                True
project_num_split           {'appl_type_code': '5', 'activity_code': 'R01'...
principal_investigators     [{'profile_id': 9871710, 'first_name': 'Shanno...
contact_pi_name                                     HAWKINS, SHA

In [50]:
(df_admins[df_admins['admin_agency'] != 'CA']).iloc[0]['agency_ic_fundings']

[{'fy': 2023,
  'code': 'CA',
  'name': 'National Cancer Institute',
  'abbreviation': 'NCI',
  'total_cost': 221501.0},
 {'fy': 2023,
  'code': 'HD',
  'name': 'Eunice Kennedy Shriver National Institute of Child Health and Human Development',
  'abbreviation': 'NICHD',
  'total_cost': 435689.0}]

In [52]:
cleaned_grants_data[cleaned_grants_data['project_num'] == '5R01HD109707-02']

,project_num,core_project_num,appl_id,fiscal_year,project_title,abstract_text,pref_terms,principal_investigators,program_officers,award_amount,...,award_notice_date,project_start_date,project_end_date,opportunity_number,api_source_search,org_name,org_city,org_state,org_country,program.program_id
1429,5R01HD109707-02,R01HD109707,10698079,2023,Targeting cellular senescence to inhibit the d...,Ovarian endometriomas are deep endometriosis l...,ARID1A gene;Acceleration;Address;Adhesions;Adu...,Shannon Michelle Hawkins,Candace M Tingen,657190,...,2023-07-17T12:07:00Z,2022-09-15T12:09:00Z,2027-06-30T12:06:00Z,PA-20-185,nofo_PA20-185,INDIANA UNIV-PURDUE UNIV AT INDIANAPOLIS,INDIANAPOLIS,IN,UNITED STATES,AllofUs


In [53]:
cleaned_grants_data.iloc[1429]

project_num                                                  5R01HD109707-02
core_project_num                                                 R01HD109707
appl_id                                                             10698079
fiscal_year                                                             2023
project_title              Targeting cellular senescence to inhibit the d...
abstract_text              Ovarian endometriomas are deep endometriosis l...
pref_terms                 ARID1A gene;Acceleration;Address;Adhesions;Adu...
principal_investigators                             Shannon Michelle Hawkins
program_officers                                            Candace M Tingen
award_amount                                                          657190
agency_ic_fundings                                                    221501
award_notice_date                                       2023-07-17T12:07:00Z
project_start_date                                      2022-09-15T12:09:00Z

In [57]:
projects = pd.read_csv('../data/processed/2023-08-30/api-gathered-2023-10-02/project.tsv', sep="\t")
projects

,project_num,core_project_num,appl_id,fiscal_year,project_title,abstract_text,pref_terms,principal_investigators,program_officers,award_amount,...,award_notice_date,project_start_date,project_end_date,opportunity_number,api_source_search,org_name,org_city,org_state,org_country,program.program_id
0,5U54CA268069-02,U54CA268069,10538588,2023,Center for Multiparametric Imaging of Tumor Im...,This project proposes an integrated suite of m...,Address;Advanced Malignant Neoplasm;Behavior;B...,"Kevin William Eliceiri, Paolo Provenzano",Steven Becker,1537616,...,2023-02-08T12:02:00Z,2021-12-09T12:12:00Z,2026-11-30T12:11:00Z,RFA-CA-21-002,nofo_RFA-CA-21-002,UNIVERSITY OF MINNESOTA,MINNEAPOLIS,MN,UNITED STATES,CellularCancerBiologyImagingResearch
1,5U54CA268072-03,U54CA268072,10684857,2023,Imaging mechanisms of metastatic tumor formati...,Project Summary In response to the RFA for a C...,3-Dimensional;Acceleration;Adaptor Signaling P...,Gaudenz Danuser,Steven Becker,1536482,...,2023-08-09T12:08:00Z,2021-09-24T12:09:00Z,2026-08-31T12:08:00Z,RFA-CA-21-002,nofo_RFA-CA-21-002,UT SOUTHWESTERN MEDICAL CENTER,DALLAS,TX,UNITED STATES,CellularCancerBiologyImagingResearch
2,5U54CA268083-02,U54CA268083,10532377,2023,Center for 3D Imaging in Cancer Cell Biology,Project Summary - Overall The mortality rates ...,3-Dimensional;Address;Advisory Committees;Advo...,"Denis Wirtz, Laura Delong Wood",Steven Becker,1667979,...,2022-12-01T12:12:00Z,2021-12-01T12:12:00Z,2026-11-30T12:11:00Z,RFA-CA-21-002,nofo_RFA-CA-21-002,JOHNS HOPKINS UNIVERSITY,BALTIMORE,MD,UNITED STATES,CellularCancerBiologyImagingResearch
3,5U54CA268084-02,U54CA268084,10539321,2023,Northwestern University Center for Chromatin N...,Overall: PROJECT SUMMARY Cancer stem cells (CS...,3-Dimensional;Address;Aftercare;Biological Ass...,"Vadim Backman, Daniela E Matei, Hao F Zhang",Steven Becker,1663001,...,2022-11-29T12:11:00Z,2021-12-10T12:12:00Z,2026-11-30T12:11:00Z,RFA-CA-21-002,nofo_RFA-CA-21-002,NORTHWESTERN UNIVERSITY,Chicago,IL,UNITED STATES,CellularCancerBiologyImagingResearch
4,1U54CA268084-01,U54CA268084,10375268,2022,Northwestern University Center for Chromatin N...,Overall: PROJECT SUMMARY Cancer stem cells (CS...,3-Dimensional;Address;Aftercare;Biological Ass...,"Vadim Backman, Daniela E Matei, Hao F Zhang",Steven Becker,1720444,...,2021-12-10T12:12:00Z,2021-12-10T12:12:00Z,2026-11-30T12:11:00Z,RFA-CA-21-002,nofo_RFA-CA-21-002,NORTHWESTERN UNIVERSITY,Chicago,IL,UNITED STATES,CellularCancerBiologyImagingResearch
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,5P50CA127003-05,P50CA127003,8135281,2011,DF/HCC SPORE in Gastrointestinal Cancer,This application represents the creation of a ...,NaN,Charles S Fuchs,Steven F Nothwehr,2185000,...,2011-09-01T12:09:00Z,2007-08-31T12:08:00Z,2013-06-30T12:06:00Z,PAR-05-156,award_P50CA127003,DANA-FARBER CANCER INST,BOSTON,MA,UNITED STATES,DanaFarberHarvardCancerCenterSPOREinGastrointe...
5568,5P50CA127003-04,P50CA127003,7879503,2010,DF/HCC SPORE in Gastrointestinal Cancer,This application represents the creation of a ...,NaN,Charles S Fuchs,Steven F Nothwehr,2409374,...,2010-08-25T12:08:00Z,2007-08-31T12:08:00Z,2012-06-30T12:06:00Z,PAR-05-156,award_P50CA127003,DANA-FARBER CANCER INST,BOSTON,MA,UNITED STATES,DanaFarberHarvardCancerCenterSPOREinGastrointe...
5569,5P50CA127003-03,P50CA127003,7662345,2009,DF/HCC SPORE in Gastrointestinal Cancer,DESCRIPTION (provided by applicant): This appl...,NaN,Charles S Fuchs,Steven F Nothwehr,2387462,...,2009-06-22T12:06:00Z,2007-08-31T12:08:00Z,2012-06-30T12:06:00Z,PAR-05-156,award_P50CA127003,DANA-FARBER CANCER INST,BOSTON,MA,UNITED STATES,DanaFarberHarvardCancerCenterSPOREinGastrointe...
5570,5P50CA127003-02,P50CA127003,7491704,2008,DF/HCC SPORE in Gastrointestinal Cancer,DESCRIPTION (provided by applicant): This appl...,NaN,Charles S Fuchs,Steven F Nothwehr,2300001,...,2008-07-18T12:07:00Z,2007-08-31T12:08:00Z,2012-06-30T12:06:00Z,PAR-05-156,award_P50CA127003,DANA-FA

In [58]:
projects[~projects['project_num'].str.contains('CA')]

,project_num,core_project_num,appl_id,fiscal_year,project_title,abstract_text,pref_terms,principal_investigators,program_officers,award_amount,...,award_notice_date,project_start_date,project_end_date,opportunity_number,api_source_search,org_name,org_city,org_state,org_country,program.program_id
1063,26114002B91019F00132-0-0-1,26114002B91,10038495,2019,Development and Expansion of Digital Tools to ...,"Earlier this year, NCI launched the Cancer Cen...",Cancer Center;Development;Evaluation;Malignant...,Sara Steinberger,NaN,5445823,...,NaN,2019-09-15T12:09:00Z,2020-09-14T12:09:00Z,NaN,award_26114002B91019F00132-0-0-1,"ICF MACRO, INC.",Reston,VA,UNITED STATES,DevelopmentandExpansionofDigitalToolstoSupport...
2695,5R01HD109707-02,R01HD109707,10698079,2023,Targeting cellular senescence to inhibit the d...,Ovarian endometriomas are deep endometriosis l...,ARID1A gene;Acceleration;Address;Adhesions;Adu...,Shannon Michelle Hawkins,Candace M Tingen,657190,...,2023-07-17T12:07:00Z,2022-09-15T12:09:00Z,2027-06-30T12:06:00Z,PA-20-185,nofo_PA20-185,INDIANA UNIV-PURDUE UNIV AT INDIANAPOLIS,INDIANAPOLIS,IN,UNITED STATES,AllofUs
3254,1R01HD109707-01,R01HD109707,10512390,2022,Targeting cellular senescence to inhibit the d...,Ovarian endometriomas are deep endometriosis l...,ARID1A gene;Address;Adhesions;Adult;Affect;Ass...,Shannon Michelle Hawkins,Candace M Tingen,657190,...,2022-09-06T12:09:00Z,2022-09-15T12:09:00Z,2027-06-30T12:06:00Z,PA-20-185,nofo_PA20-185,INDIANA UNIV-PURDUE UNIV AT INDIANAPOLIS,INDIANAPOLIS,IN,UNITED STATES,AllofUs
4199,3D43TW009340-12S1,D43TW009340,10852359,2023,UJMT FGHF Launching Future Leaders in Global H...,PROJECT SUMMARY In this supplemental applicati...,Funding;Future;Health Insurance;National Insti...,"William Checkley, Benjamin H. Chi, Kofi A Kond...",Unja Hayes,217611,...,2023-06-27T12:06:00Z,2012-04-04T12:04:00Z,2027-06-30T12:06:00Z,PA-20-272,nofo_PA20-272,UNIV OF NORTH CAROLINA CHAPEL HILL,CHAPEL HILL,NC,UNITED STATES,AllofUs
4223,3P20GM139760-03S1,P20GM139760,10724723,2023,Novel Dual-Modality Treatment of Breast Cancer...,Project Summary Bone is one of the top sites f...,4T1;Acceleration;Address;Age Months;Aging;Anim...,Dawn M Elliott,Irina N Krasnova,306774,...,2023-05-22T12:05:00Z,2021-02-15T12:02:00Z,2026-01-31T12:01:00Z,PA-20-272,nofo_PA20-272,UNIVERSITY OF DELAWARE,NEWARK,DE,UNITED STATES,AllofUs
4238,3D43TW010540-07S1,D43TW010540,10854174,2023,Global Health Equity Scholars Program,PROGRAM SUMMARY The objective of our LAUNCH pr...,Address;Africa;Arizona;Asia;California;Career ...,"Michele Barry, Eva Harris, Albert Icksang Ko, ...",Unja Hayes,243423,...,2023-08-04T12:08:00Z,2017-07-01T12:07:00Z,2027-06-30T12:06:00Z,PA-20-272,nofo_PA20-272,YALE UNIVERSITY,NEW HAVEN,CT,UNITED STATES,AllofUs
4269,3U01AG077920-04S1,U01AG077920,10831334,2023,The intestinal microbiome contribution to colo...,Project Summary This application is being subm...,Address;Age;Aging;Alleles;American;ApcMin/+ mi...,William Mallory Grady,Max Guo,183278,...,2023-09-04T12:09:00Z,2021-09-30T12:09:00Z,2026-08-31T12:08:00Z,PA-20-272,nofo_PA20-272,FRED HUTCHINSON CANCER CENTER,Seattle,WA,UNITED STATES,AllofUs
4284,3P20GM109095-10S2,P20GM109095,10722845,2023,Role of mechanical stress in mitigating chemot...,PROJECT SUMMARY/ABSTRACT The Center of Biomedi...,Administrative Supplement;Age;Biology;Breast C...,Julia Thom Oxford,Crina Frincu,280294,...,2023-09-05T12:09:00Z,2014-08-01T12:08:00Z,2024-05-31T12:05:00Z,PA-20-272,nofo_PA20-272,BOISE STATE UNIVERSITY,BOISE,ID,UNITED STATES,AllofUs
4332,3D43TW010543-07S1,D43TW010543,10854210,2023,Partnership for Global Health Research Trainin...,The HBNU Fogarty Global Health Training Progra...,Acquired Immunodeficiency Syndrome;Award;Biome...,"Wafaie W Fawzi, Davidson Howes Hamer, Claudia ...",Unja Hayes,275280,...,2023-08-12T12:08:00Z,2017-07-01T12:07:00Z,2027-06-30T12:06:00Z,PA-20-272,nofo_PA20-272,HARVARD SCHOOL OF PUBLIC HEALTH,BOSTON,MA,UNITED STATES,AllofUs
4333,3U01AG073204-03S1,U01AG073204,1086295